In [1]:
import json
import pandas as pd

from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB
from pprint import pprint
from string import punctuation
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from unidecode import unidecode
from ethnicity_features import Selector, WordCount, NameLength, FirstLast, DictFirstNameFeatures
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [2]:
class EP:
    
    def __init__(self, ethnicity):  
        
        assert ethnicity in ('iranian polish italian german japanese vietnamese'
                             ' portuguese fijian indian croatian greek'
                             ' serbian spanish korean thai turkish' 
                             ' english cambodian russian').split(), f'incorrect ethnicity: {ethnicity}!'
        
        datafile = f'/Users/ik/Data/names/training-{ethnicity}.csv'
        self.target_col = f'is_{ethnicity}'
        
        self.data = pd.read_csv(datafile).sample(frac=1.)
        self.data['full_name'] = self.data['full_name'].apply(lambda _: unidecode(_))
        
        assert self.target_col in self.data.columns, f'there is no {self.target_col} in data file!'  
        
        try:
            self.NODOUBT_FIRST_NAMES = list({line.strip() for line in open(f'/Users/ik/Data/names/real_{ethnicity}.txt','r').readlines() if line.strip()})
        except:
            self.NODOUBT_FIRST_NAMES = []
        print(self.NODOUBT_FIRST_NAMES)
        
    def add_nodoubt_names(self):
        
        print(f'adding {len(self.NODOUBT_FIRST_NAMES)} first names..')
        
        self.data = pd.concat([self.data, pd.DataFrame({'full_name': self.NODOUBT_FIRST_NAMES, 
                                                        self.target_col: [1]*len(self.NODOUBT_FIRST_NAMES)})], 
                              ignore_index=True).sample(frac=1.)
        return self

In [3]:
# class Selector(BaseEstimator, TransformerMixin):
#     """
#     select a columns from a data frame and return as a list
#     """
#     def __init__(self, col_name):
#         self.col_name = col_name
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         return '_start_' + x[self.col_name] + '_end_'

# class WordCount(BaseEstimator, TransformerMixin):
#     """
#     select a columns from a data frame and return as a list
#     """
#     def __init__(self):
#         pass
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         res = x.apply(lambda _: len(_.split())).values.reshape(x.shape[0],1)
#         return res

# class NameLength(BaseEstimator, TransformerMixin):
#     """
#     return the length of the full name
#     """
#     def __init__(self):
#         pass
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         res = x.str.len().values.reshape(x.shape[0],1)
#         return res

# class FirstLast(BaseEstimator, TransformerMixin):
#     """
#     is the first word longer than the last one
#     """
#     def __init__(self):
#         pass
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         res = x.apply(lambda _: np.argmax([len(p) for i, p in enumerate(_.split()) if i in [0,len(_.split())-1]])).values.reshape(x.shape[0],1)
#         return res

In [ ]:
if __name__ == '__main__':
    
    ic = EP('fijian').add_nodoubt_names()
    
    # split into the training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(ic.data.drop([ic.target_col], axis=1), 
                                                        ic.data[ic.target_col],  test_size=0.3, 
                                                        random_state=391, stratify=ic.data[ic.target_col])
    

    pipe = Pipeline([('select_fullname', Selector('full_name')),  # df with 1 column
                     ('features', FeatureUnion(
                         [('char_level', CountVectorizer(strip_accents='ascii', analyzer='char', 
                                                          ngram_range=(2, 4))),
                          ('word_level', CountVectorizer(strip_accents='ascii', analyzer='word',
                                                       ngram_range=(1,3))),
                          ('word_count', WordCount()),
                          ('dict_fname', DictFirstNameFeatures(ic.NODOUBT_FIRST_NAMES)),
                          ('full_name_length', NameLength()),
                          ('firstlast', FirstLast())],
                              transformer_weights={
                                            'char_level': 0.8,
                                            'word_level': 0.5,
                                            'word_lengths': 1.0,})),
                     ('normalise', Normalizer()),
                     ('pca', TruncatedSVD(n_components=120)),
                     ('clf', VotingClassifier(estimators=[('sgd', SGDClassifier(max_iter=1000)), 
                                                           ('rf', RandomForestClassifier(class_weight='balanced')),
                                                            ('svc', SVC(C=0.0001)),
                                                             ('abst', AdaBoostClassifier(n_estimators=200)),
                                                             ('gb', GradientBoostingClassifier(n_estimators=200))], 
                                                                    voting='hard'))])
    
    param_grid = {
#         'features__transformer_weights': [{'char_level': 0.2,
#                                                      'word_level': 0.1,
#                                                      'word_count':0.4,
#                                                     'full_name_length': 0.5}, 
#                                                    {'char_level': 0.9,
#                                                      'word_level': 0.2,
#                                                      'word_lengths':0.9,
#                                                    'full_name_length': 0.3},
#                                                    {'char_level': 0.9,
#                                                      'word_level': 0.3,
#                                                      'word_count':0.2,
#                                                    'full_name_length': 0.7}],
                    'clf__sgd__loss': ['modified_huber'], # also 'hinge', 'perceptron', 'log'
                    'clf__sgd__penalty': ['l2'],  # 'elasticnet', 'l1'
                    'clf__rf__n_estimators': [100, 200],
                    'clf__rf__max_depth': [None, 3],
                    'clf__svc__C': [1e-6, 1e-5, 1e-4, 1e-2, 1.],  # default is 1.
                 'pca__n_components': [30, 120, 400]}
    
    grid_search = GridSearchCV(pipe, param_grid=param_grid, verbose=10)
    
    grid_search.fit(X_train, y_train)

['tarlae', 'asenaca', 'makereta', 'keresi', 'siteri', 'josivini', 'joni', 'alexade', 'alitia', 'karalaini', 'iloilo', 'esiteri', 'elisapeci', 'sovaia', 'miliana', 'iloilovatu', 'semesa', 'apenisa', 'kalara', 'selai', 'ifereimi', 'walota', 'kona', 'maciu', 'elenoa', 'nete', 'lelea', 'ilikimi', 'ilisapeci', 'yasi', 'nimlesh']
adding 31 first names..
Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06, pca__n_components=30 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06, pca__n_components=30, score=0.8379888268156425, total=   1.4s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06, pca__n_components=30 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.9s remaining:    0.0s


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06, pca__n_components=30, score=0.8687150837988827, total=   1.3s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06, pca__n_components=30 


...

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0, pca__n_components=120, score=0.848314606741573, total=   3.6s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0, pca__n_components=400 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0, pca__n_components=400, score=0.88268156424581, total=  11.0s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0, pca__n_components=400 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
print(classification_report(y_test, grid_search.predict(X_test)))

In [ ]:
test_data = pd.DataFrame({'full_name': ['han walota', 'xi','babak', 'igor souza', 'yash patel', 'david perron',
                                        'andrzej woods','otto floms', 'suzuki', 'iloilovatu', 
                                        'edilson', 'kawasaki','silva', 'james tran', 
                                        'bo nguyen', 'zlatan ibrahimovic', 'ronaldo barbosa', 'natalia klanchinkova', 
                                        'bob', 'keresi right','kalara',  'kona reed', 'andreas vlachos', 'con', 'nemanja vidic']})

test_data['prediction'] = grid_search.predict(test_data)
test_data['prediction'] = test_data['prediction'].apply(lambda _: 'yes' if _ else 'no')

In [ ]:
test_data

In [ ]:
grid_search.best_params_

In [ ]:
for t in zip(X_test.values, grid_search.predict(X_test), y_test.values):
    if t[1] != t[2]:
        print(f'{t[0][0].upper()} predicted {t[1]} but it is {t[2]}')

In [ ]:
grid_search.best_estimator_

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(grid_search.best_estimator_, 'fijian_model.pkl', compress = 1)

In [ ]:
m = joblib.load('russian_model.pkl')